# 特征提取

### 基本文本处理技能

##### 分词算法设计中的几个基本原则：

1、颗粒度越大越好：用于进行语义分析的文本分词，要求分词结果的颗粒度越大，即单词的字数越多，所能表示的含义越确切，如：“公安局长”可以分为“公安 局长”、“公安局 长”、“公安局长”都算对，但是要用于语义分析，则“公安局长”的分词结果最好（当然前提是所使用的词典中有这个词）

2、切分结果中非词典词越少越好，单字字典词数越少越好，这里的“非词典词”就是不包含在词典中的单字，而“单字字典词”指的是可以独立运用的单字，如“的”、“了”、“和”、“你”、“我”、“他”。例如：“技术和服务”，可以分为“技术 和服 务”以及“技术 和 服务”，但“务”字无法独立成词（即词典中没有），但“和”字可以单独成词（词典中要包含），因此“技术 和服 务”有1个非词典词，而“技术 和 服务”有0个非词典词，因此选用后者。

3、总体词数越少越好，在相同字数的情况下，总词数越少，说明语义单元越少，那么相对的单个语义单元的权重会越大，因此准确性会越高。

#### 分词的概念

分词的正向最大: 正向即从前往后取词，从7->1，每次减一个字，直到词典命中或剩下1个单字。

分词的逆向最大: 逆向即从后往前取词，其他逻辑和正向相同。

分词双向最大匹配法: 正向最大匹配法和逆向最大匹配法，都有其局限性，我举得例子是正向最大匹配法局限性的例子，逆向也同样存在（如：长春药店，逆向切分为“长/春药店”），因此有人又提出了双向最大匹配法，双向最大匹配法。即，两种算法都切一遍，然后根据大颗粒度词越多越好，非词典词和单字词越少越好的原则，选取其中一种分词结果输出。

#### 词、字符频率统计

使用Python中的collections.Counter模块。
词频率统计：第一步分词，然后根据分词后的结果进行词频率统计。
基于jieba的实现代码：

In [10]:
# 官方文档 https://github.com/fxsjy/jieba
import jieba
from collections import Counter
seg_list=jieba.lcut('今天是计划安排的第五天。', cut_all=False)
print("Default Mode:" + "/".join(seg_list))

# lcut 返回列表

WordCount = Counter(seg_list)
print(WordCount)

Default Mode:今天/是/计划/安排/的/第五天/。
Counter({'今天': 1, '是': 1, '计划': 1, '安排': 1, '的': 1, '第五天': 1, '。': 1})


In [11]:
txt = '今天是计划安排的第五天。'
wordcount = Counter(txt)

print(wordcount)

Counter({'天': 2, '今': 1, '是': 1, '计': 1, '划': 1, '安': 1, '排': 1, '的': 1, '第': 1, '五': 1, '。': 1})


### 语言模型

n-gram模型（考虑句子中单词之间的顺序）

当n取1、2、3时，n-gram模型分别称为unigram、bigram、trigram语言模型

unigram一元分词，把句子分成一个一个的汉字

bigram二元分词，把句子从头到尾每两个字组成一个词语

trigram三元分词，把句子从头到尾每三个字组成一个词语


In [ ]:
def unigram(dic, train_data_dic):
    all_value_un = 0 
    # To get all counts
    for i in dic:
        all_value_un += dic[i]
    # To get the train data dictionary
    for i in dic:
        value = dic[i] / all_value_un
        train_data_dic[i] = value
    return train_data_dic

# To get the bigram words: use 'zip' to structure
def bigrams(words):
    return zip(words, words[1:])

### 文本矩阵化

In [ ]:
import jieba
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
 
 
# 读取停用词
def read_stopword(filename):
    stopword = []
    fp = open(filename, 'r')
    for line in fp.readlines():
        stopword.append(line.replace('\n', ''))
    fp.close()
    return stopword
 
 
# 切分数据，并删除停用词
def cut_data(data, stopword):
    words = []
    for content in data['content']:
        word = list(jieba.cut(content))
        for w in list(set(word) & set(stopword)):
            while w in word:
                word.remove(w)
        words.append(' '.join(word))
    data['content'] = words
    return data
 
 
# 获取单词列表
def word_list(data):
    all_word = []
    for word in data['content']:
        all_word.extend(word)
    all_word = list(set(all_word))
    return all_word
 
 
# 计算文本向量
def text_vec(data):
    count_vec = CountVectorizer(max_features=300, min_df=2)
    count_vec.fit_transform(data['content'])
    fea_vec = count_vec.transform(data['content']).toarray()
    return fea_vec
 
 
if __name__ == '__main__':
    data = pd.read_csv('./cnews/test.txt', names=['title', 'content'], sep='\t')  # (10000, 2)
 
    stopword = read_stopword('./cnews/stopword.txt')
    data = cut_data(data, stopword)
 
    fea_vec = text_vec(data)
    print(fea_vec)

In [3]:
conda list 


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip list

Package              Version  
-------------------- ---------
absl-py              0.7.1    
astor                0.8.0    
backcall             0.1.0    
certifi              2019.6.16
colorama             0.4.1    
cycler               0.10.0   
decorator            4.4.0    
gast                 0.2.2    
google-pasta         0.1.7    
grpcio               1.21.1   
h5py                 2.9.0    
ipykernel            5.1.1    
ipython              7.5.0    
ipython-genutils     0.2.0    
jedi                 0.13.3   
jieba                0.39     
joblib               0.13.2   
jupyter-client       5.2.4    
jupyter-core         4.4.0    
Keras-Applications   1.0.8    
Keras-Preprocessing  1.1.0    
kiwisolver           1.1.0    
Markdown             3.1.1    
matplotlib           3.1.0    
mkl-fft              1.0.12   
mkl-random           1.0.2    
mkl-service          2.0.2    
numpy                1.16.4   
parso                0.4.0    
pickleshare          0.7.5    
pip     